In [1]:
import pandas as pd
import unicodedata
import numpy as np
import os

In [ ]:


def leer_nombres_archivos():
    """Obtiene una lista de archivos en el directorio actual"""
    archivos = [archivo for archivo in os.listdir(os.getcwd())]
    return archivos

def tratar_texto(texto):
    """Normaliza cadenas eliminando tildes y caracteres especiales"""
    if isinstance(texto, str):
        texto_normalizado = unicodedata.normalize('NFD', texto)
        texto_sin_tildes = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
        return texto_sin_tildes    
    else:
        return texto  # Si no es texto, devuelve el valor original

def diccionario_archivos_excel():
    """Crea un diccionario con los datos de los archivos Excel procesados"""
    archivos_excel = {}
    for archivo in leer_nombres_archivos():
        # Solo procesar archivos con extensión .xlsx
        if archivo.endswith(".xlsx"):
            ruta_completa = os.path.join(os.getcwd(), archivo)
            try:
                # Cargar el archivo Excel
                df = pd.read_excel(
                    ruta_completa,
                    skiprows=7,
                    usecols="D:K",
                    engine="openpyxl"
                )
                # Renombrar columnas específicas
                df = df.rename(columns={
                    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
                    "FECHA POSESIÓN": "FECHA_POSESION",
                    "FECHA BAJA": "FECHA_BAJA",
                    "CÓDIGO INE": "CODIGO_NIE"
                })
                # Convertir fechas a formato datetime sin horas, minutos ni segundos
                

                # Aplicar tratamiento de texto a cada celda
                df_sin_tildes = df.map(tratar_texto)
                # Guardar el DataFrame en el diccionario
                nombre_base = os.path.splitext(archivo)[0]
                archivos_excel[nombre_base] = df_sin_tildes
                print(f"Datos del archivo {archivo} cargados con éxito.")
            except Exception as e:
                print(f"Error al procesar el archivo {archivo}: {e}")
    return archivos_excel



# Procesar los archivos Excel
archivos_excel = diccionario_archivos_excel()

# Guardar los resultados en un único archivo Excel
archivo_salida = "excel_completo.xlsx"
with pd.ExcelWriter(archivo_salida, engine="openpyxl") as writer:
    for nombre_hoja, df in archivos_excel.items():
        # Guardar cada DataFrame en una hoja separada
        df.to_excel(writer, sheet_name=nombre_hoja, index=False)

print(f"Archivo combinado guardado como {archivo_salida}")


In [ ]:
ruta_archivo = "excel_completo.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
    
# Combinar datos de todas las hojas en un solo DataFrame
df_legislaturas = pd.concat(hojas.values(), ignore_index=True)

# Para tema de formateo de fechas para las tablas FECHA_POSESION y FECHA_BAJA
df_legislaturas["FECHA_POSESION"] = pd.to_datetime(df_legislaturas["FECHA_POSESION"], errors="coerce", dayfirst=True).dt.date
df_legislaturas["FECHA_BAJA"] = pd.to_datetime(df_legislaturas["FECHA_BAJA"], errors="coerce", dayfirst=True).dt.date

# Contar cuántas veces aparece cada alcalde
conteo_alcaldes = df_legislaturas.groupby("NOMBRE").size()

# Filtrar solo los alcaldes que aparecen más de una vez
alcaldes_repetidos = conteo_alcaldes[conteo_alcaldes > 1].index

# Filtrar el DataFrame original con estos alcaldes y se usará de base
df_repetidos = df_legislaturas[df_legislaturas["NOMBRE"].isin(alcaldes_repetidos)]

# Agrupamos por nombre en una columna y egrContar cuántas veces ha sido elegido cada alcalde 
df_legislaturas_repetidos = df_repetidos.groupby(["NOMBRE","MUNICIPIO"])["FECHA_POSESION"].apply(list).reset_index()
df_legislaturas_repetidos["REELEGIDO"] = df_legislaturas_repetidos["FECHA_POSESION"].apply(len)
df_legislaturas_repetidos = df_legislaturas_repetidos[df_legislaturas_repetidos["REELEGIDO"] > 1]

df_legislaturas_repetidos.sort_values(by="REELEGIDO",ascending=False)

2.En esta burbuja vamos a saber lo siguiente:

De repetir mandato. Qué partidos políticos tiveron máis éxito? Qué alcaldes mudaron de partido?

In [ ]:
partidos_exitosos = df_repetidos.groupby("LISTA")["NOMBRE"].count().reset_index() 
partidos_exitosos.columns = ["LISTA", "VECES_VOTADOS"]

partidos_exitosos = partidos_exitosos.sort_values(by="VECES_VOTADOS", ascending=False)
print(partidos_exitosos)

alcaldes_cambiantes = df_repetidos.groupby("NOMBRE")["LISTA"].nunique().reset_index()
alcaldes_cambiantes["REELEGIDO"] = df_repetidos["LISTA"]
alcaldes_cambiantes.columns = ["NOMBRE", "PARTIDOS_DISTINTOS","REELEGIDO"]

alcaldes_cambiantes = alcaldes_cambiantes[alcaldes_cambiantes["PARTIDOS_DISTINTOS"] > 1]
alcaldes_cambiantes = alcaldes_cambiantes.sort_values(by="PARTIDOS_DISTINTOS", ascending=False)
print(alcaldes_cambiantes)

3.En esta burbuja vamos a calular lo siguiente:

Cantos alcaldes cesaron antes de tempo?

In [ ]:
df_cesados = df_legislaturas[df_legislaturas["FECHA_BAJA"].notna()]
df_cesados_alcaldes_totales = df_cesados.groupby("FECHA_BAJA")["NOMBRE"].unique().count()


print(f"Número de alcaldes que se fueron antes de tiempo: {df_cesados_alcaldes_totales}")

4.En esta burbuja vamos a conseguir lo siguiente:

Cal é o soldo máis alto e de que alcaldes?

In [ ]:

ruta_archivo_alcaldes = "Alcaldes_Mandato_2019_2023.xlsx"
ruta_archivo_retribuciones = "tabula-retribuciones_alcaldes_2021.csv"
df_alcaldes = pd.read_excel(
    ruta_archivo_alcaldes,
    skiprows=7,
    usecols="D:K",
    engine="openpyxl"
)
df_alcaldes = df_alcaldes.rename(columns={
    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
    "FECHA POSESIÓN": "FECHA_POSESION",
    "FECHA BAJA": "FECHA_BAJA",
    "CÓDIGO INE": "CODIGO_NIE"
})
df_alcaldes = df_alcaldes.map(tratar_texto)
df_retribuciones = pd.read_csv(ruta_archivo_retribuciones, skiprows=1)
df_retribuciones = df_retribuciones.rename(columns={"CCAA": "COMUNIDAD_AUTONOMA",
                                                    "DEDICACIÓN": "REGIMEN_DEDICACION",
                                                    "PERCIBIDO": "TOTAL_PERCIBIDO",
                                                    "AYUNTAMIENTO": "MUNICIPIO"
                                                    }
                                           )
df_retribuciones = df_retribuciones.map(tratar_texto)
df_retribuciones_alcaldes = pd.merge(df_alcaldes, df_retribuciones, on="MUNICIPIO", how="inner")

df_retribuciones_alcaldes["TOTAL_PERCIBIDO"] = df_retribuciones_alcaldes["TOTAL_PERCIBIDO"].astype(str).str.replace(".","").str.replace(",",".").astype(float)

df_retribuciones_alcaldes = df_retribuciones_alcaldes[["TOTAL_PERCIBIDO","NOMBRE","MUNICIPIO","COMUNIDAD_AUTONOMA_x","LISTA"]]
df_retribuciones_alcaldes.sort_values(by="TOTAL_PERCIBIDO", ascending=False).head(10)

5.En esta burbuja vamos a conseguir lo siguiente:

Relaciona os soldos altos e baixos coa "probabilidade" de repetir mandato.


In [ ]:
df_final = df_legislaturas_repetidos.merge(df_retribuciones_alcaldes, on="NOMBRE", how="inner")
df_final = df_final[df_final["TOTAL_PERCIBIDO"] > 0.00]
df_final.sort_values(by="TOTAL_PERCIBIDO", ascending=False) 

In [ ]:
#BONUS
df_cesados_agrupados = df_cesados.groupby("NOMBRE").size().reset_index(name="COMUNIDAD_AUTONO")

df_cesados_agrupados = df_cesados_agrupados.sort_values(by="COMUNIDAD_AUTONO", ascending=False)
df_cesados_agrupados